In [1]:
!pip install -q scikit-learn scikit-survival

In [2]:
import os
import pandas as pd
import numpy as np
import numpy as np
import pickle
import subprocess
from datetime import datetime
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import GridSearchCV
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.metrics import concordance_index_censored

In [3]:
def load_dataset(remote_path):
    import os
    import pandas as pd

    local_filename = os.path.basename(remote_path)  # extracts just 'dfTrain.csv'
    !dx download "{remote_path}" --overwrite
    dataset = pd.read_csv(local_filename)
    return dataset

dfTrain = load_dataset("SADIQ/30-04/dfTrain.csv")
dfTest = load_dataset("SADIQ/30-04/dfTest.csv")

[===========================================================>] Completed 967,397,153 of 967,397,153 bytes (100%) /opt/notebooks/dfTrain.csvsv
[===========================================================>] Completed 968,855,153 of 968,855,153 bytes (100%) /opt/notebooks/dfTest.csvsv


In [4]:
print(len(dfTest))
dfTest = dfTest.drop(dfTest[
    ((dfTest['CVD_followup'] >= 10) & (dfTest['CVD_followup'] <= 12)) &
    (dfTest['CVD_status'] == True)
].index)
print(len(dfTest))

120495
118376


In [5]:
print(len(dfTest))
dfTest = dfTest[dfTest['CVD_followup']>= 0]
len(dfTest)

118376


118376

In [6]:
dfTrain = dfTrain.drop(columns = ['qrisk_Age.at.recruitment','qrisk_Sex_0','qrisk_Sex_1'])
dfTest = dfTest.drop(columns = ['qrisk_Age.at.recruitment','qrisk_Sex_0','qrisk_Sex_1'])

In [7]:
mrmr = ['ts_Vascular.heart.problems.diagnosed.by.doctor...Instance.0_neg7', 'ts_Age.at.menopause..last.menstrual.period....Instance.0_Middle.third', 'ts_Difficulty.not.smoking.for.1.day...Instance.0_4', 'ts_Vascular.heart.problems.diagnosed.by.doctor...Instance.0_2', 'ts_Other.serious.medical.condition.disability.diagnosed.by.doctor...Instance.0_0', 'ts_Age.completed.full.time.education...Instance.0_Lower.third', 'ts_Combined.sex.medication_3', 'ts_Ever.taken.oral.contraceptive.pill...Instance.0_1', 'ts_Chest.pain.due.to.walking.ceases.when.standing.still...Instance.0_1', 'ts_Mother.still.alive...Instance.0_0', 'ts_Medication.for.pain.relief..constipation..heartburn...Instance.0_1', 'ts_Vascular.heart.problems.diagnosed.by.doctor...Instance.0_1', 'ts_Usual.walking.pace...Instance.0_1', 'ts_Combined.sex.medication_1', 'ts_Ever.used.hormone.replacement.therapy..HRT....Instance.0_0', 'ts_Current.employment.status...Instance.0_1', 'ts_Combined.sex.medication_2', 'ts_Mouth.teeth.dental.problems...Instance.0_6', 'ts_Ever.had.hysterectomy..womb.removed....Instance.0_0', 'ts_Illnesses.of.siblings...Instance.0_1', 'ts_Chest.pain.or.discomfort.walking.normally...Instance.0_1', 'ts_Long.standing.illness..disability.or.infirmity...Instance.0_0', 'ts_Vascular.heart.problems.diagnosed.by.doctor...Instance.0_4', 'ts_Father.still.alive...Instance.0_0', 'ts_Age.angina.diagnosed...Instance.0_Upper.third', 'ts_Ever.had.cervical.smear.test...Instance.0_1', 'ts_Started.insulin.within.one.year.diagnosis.of.diabetes...Instance.0_0', 'ts_Age.angina.diagnosed...Instance.0_Middle.third', 'ts_Current.employment.status...Instance.0_2', 'ts_Usual.walking.pace...Instance.0_3', 'ts_Chest.pain.or.discomfort...Instance.0_0', 'ts_Combined.sex.medication_neg7', 'ts_Bilateral.oophorectomy..both.ovaries.removed....Instance.0_0', 'ts_Attendance.disability.mobility.allowance...Instance.0_neg7', 'ts_Taking.other.prescription.medications...Instance.0_0', 'ts_Qualifications...Instance.0_neg7', 'ts_Relative.age.of.first.facial.hair...Instance.0_2', 'ts_Long.standing.illness..disability.or.infirmity...Instance.0_1', 'ts_Job.involves.heavy.manual.or.physical.work...Instance.0_1', 'ts_Overall.health.rating...Instance.0_4', 'ts_Diabetes.diagnosed.by.doctor...Instance.0_0', 'ts_Smoking.status...Instance.0_0', 'ts_Vascular.heart.problems.diagnosed.by.doctor...Instance.0_3', 'ts_Relative.age.voice.broke...Instance.0_2', 'ts_Mother.s.age...Instance.0_Lower.third', 'ts_Chest.pain.or.discomfort...Instance.0_1', 'ts_Job.involves.shift.work...Instance.0_1', 'ts_Diabetes.diagnosed.by.doctor...Instance.0_1', 'ts_Had.menopause...Instance.0_0', 'ts_Taking.other.prescription.medications...Instance.0_1', 'ts_Age.high.blood.pressure.diagnosed...Instance.0_Upper.third', 'ts_Illnesses.of.siblings...Instance.0_neg17', 'ts_Overall.health.rating...Instance.0_3', 'ts_Nap.during.day...Instance.0_1', 'ts_Age.angina.diagnosed...Instance.0_Lower.third', 'ts_Ever.had.bowel.cancer.screening...Instance.0_0', 'ts_Number.of.children.fathered...Instance.0_Upper.third', 'ts_Age.heart.attack.diagnosed...Instance.0_Upper.third', 'ts_Chest.pain.or.discomfort.when.walking.uphill.or.hurrying...Instance.0_1', 'ts_Illnesses.of.father...Instance.0_1', 'ts_Average.total.household.income.before.tax...Instance.0_1', 'ts_Age.when.periods.started..menarche....Instance.0_Middle.third', 'ts_Time.spent.watching.television..TV....Instance.0', 'ts_Attendance.disability.mobility.allowance...Instance.0_3', 'ts_Frequency.of.travelling.from.home.to.job.workplace...Instance.0_Middle.third', 'ts_Current.tobacco.smoking...Instance.0_1', 'ts_Menstruating.today...Instance.0_0', 'ts_Wheeze.or.whistling.in.the.chest.in.last.year...Instance.0_1', 'ts_Hair.balding.pattern...Instance.0_3', 'ts_Age.high.blood.pressure.diagnosed...Instance.0_Middle.third', 'ts_Other.serious.medical.condition.disability.diagnosed.by.doctor...Instance.0_1', 'ts_Age.heart.attack.diagnosed...Instance.0_Lower.third', 'ts_How.are.people.in.household.related.to.participant...Instance.0_2', 'ts_Hearing.difficulty.problems...Instance.0_1', 'ts_Ever.had.stillbirth..spontaneous.miscarriage.or.termination...Instance.0_0', 'ts_Medication.for.pain.relief..constipation..heartburn...Instance.0_5', 'ts_Ever.had.breast.cancer.screening...mammogram...Instance.0_0', 'ts_Attendance.disability.mobility.allowance...Instance.0_2', 'ts_Ever.had.bowel.cancer.screening...Instance.0_1', 'ts_Medication.for.pain.relief..constipation..heartburn...Instance.0_neg7', 'ts_Own.or.rent.accommodation.lived.in...Instance.0_2', 'ts_Hair.balding.pattern...Instance.0_4', 'ts_Type.of.tobacco.previously.smoked...Instance.0_1', 'ts_Age.heart.attack.diagnosed...Instance.0_Middle.third', 'ts_Illnesses.of.mother...Instance.0_1', 'ts_Wheeze.or.whistling.in.the.chest.in.last.year...Instance.0_0', 'ts_Age.started.oral.contraceptive.pill...Instance.0_Lower.third', 'ts_Types.of.physical.activity.in.last.4.weeks...Instance.0_2', 'ts_Job.involves.mainly.walking.or.standing...Instance.0_1', 'ts_Major.dietary.changes.in.the.last.5.years...Instance.0_1', 'ts_Time.since.last.prostate.specific.antigen..PSA..test...Instance.0_Middle.third', 'ts_Number.of.live.births...Instance.0_Lower.third', 'ts_Overall.health.rating...Instance.0_1', 'ts_Mother.s.age...Instance.0_Middle.third', 'ts_Hearing.difficulty.problems.with.background.noise...Instance.0_1', 'ts_Qualifications...Instance.0_2', 'ts_Age.diabetes.diagnosed...Instance.0_Upper.third', 'ts_Age.diabetes.diagnosed...Instance.0_Lower.third', 'ts_Age.started.oral.contraceptive.pill...Instance.0_Middle.third', 'ts_Age.stroke.diagnosed...Instance.0_Upper.third', 'ts_Time.spend.outdoors.in.summer...Instance.0', 'ts_Age.deep.vein.thrombosis..DVT..blood.clot.in.leg..diagnosed...Instance.0_Upper.third', 'ts_Exposure.to.tobacco.smoke.at.home...Instance.0_Middle.third', 'ts_Transport.type.for.commuting.to.job.workplace...Instance.0_1', 'ts_Number.of.children.fathered...Instance.0_Middle.third', 'ts_Age.stroke.diagnosed...Instance.0_Middle.third', 'ts_Length.of.time.at.current.address...Instance.0', 'ts_Knee.pain.for.3..months...Instance.0_1', 'ts_Hearing.difficulty.problems...Instance.0_0', 'ts_Qualifications...Instance.0_1', 'ts_Nap.during.day...Instance.0_3', 'ts_Age.at.first.live.birth...Instance.0_Upper.third', 'ts_Age.diabetes.diagnosed...Instance.0_Middle.third', 'ts_Ever.stopped.smoking.for.6..months...Instance.0_0', 'ts_Father.s.age...Instance.0_Middle.third', 'ts_Use.of.sun.uv.protection...Instance.0_1', 'ts_Most.recent.bowel.cancer.screening...Instance.0_Middle.third', 'ts_Time.spent.outdoors.in.winter...Instance.0', 'ts_Current.employment.status...Instance.0_4', 'ts_Combined.sex.major.operations_1', 'ts_Time.since.last.prostate.specific.antigen..PSA..test...Instance.0_Lower.third', 'ts_Hearing.difficulty.problems.with.background.noise...Instance.0_0', 'ts_Qualifications...Instance.0_3', 'ts_Number.in.household...Instance.0_Upper.third', 'ts_Mouth.teeth.dental.problems...Instance.0_neg7', 'ts_Age.when.last.used.oral.contraceptive.pill...Instance.0_Middle.third', 'ts_Ever.tried.to.stop.smoking...Instance.0_1', 'ts_Length.of.menstrual.cycle...Instance.0_Middle.third', 'ts_Attendance.disability.mobility.allowance...Instance.0_1', 'ts_Back.pain.for.3..months...Instance.0_1', 'ts_Nap.during.day...Instance.0_2', 'ts_Wears.glasses.or.contact.lenses...Instance.0_0', 'ts_Blood.clot..DVT..bronchitis..emphysema..asthma..rhinitis..eczema..allergy.diagnosed.by.doctor...Instance.0_6', 'ts_Mother.s.age.at.death...Instance.0_Middle.third', 'ts_Distance.between.home.and.job.workplace...Instance.0_Middle.third', 'ts_Types.of.physical.activity.in.last.4.weeks...Instance.0_neg7', 'ts_Overall.health.rating...Instance.0_2', 'ts_Ever.stopped.smoking.for.6..months...Instance.0_1', 'ts_Age.when.last.used.oral.contraceptive.pill...Instance.0_Lower.third', 'ts_Daytime.dozing...sleeping...Instance.0_1', 'ts_Pain.type.s..experienced.in.last.month...Instance.0_6', 'ts_Chest.pain.or.discomfort.walking.normally...Instance.0_0', 'ts_Bread.intake...Instance.0', 'ts_Average.total.household.income.before.tax...Instance.0_4', 'ts_Time.since.last.menstrual.period...Instance.0_Middle.third', 'ts_Types.of.physical.activity.in.last.4.weeks...Instance.0_1', 'ts_Hearing.aid.user...Instance.0_1', 'ts_Age.high.blood.pressure.diagnosed...Instance.0_Lower.third', 'ts_Combined.sex.major.operations_0', 'ts_Chest.pain.due.to.walking.ceases.when.standing.still...Instance.0_0', 'ts_Wears.glasses.or.contact.lenses...Instance.0_1', 'ts_Years.since.last.cervical.smear.test...Instance.0_Lower.third', 'ts_Number.of.cigarettes.currently.smoked.daily..current.cigarette.smokers....Instance.0_Upper.third', 'ts_Length.of.working.week.for.main.job...Instance.0_Middle.third', 'ts_Age.when.last.used.oral.contraceptive.pill...Instance.0_Upper.third', 'ts_Never.eat.eggs..dairy..wheat..sugar...Instance.0_5', 'ts_Illnesses.of.father...Instance.0_neg17', 'ts_Qualifications...Instance.0_4', 'ts_Started.insulin.within.one.year.diagnosis.of.diabetes...Instance.0_1', 'ts_Pain.type.s..experienced.in.last.month...Instance.0_7', 'ts_Time.since.last.menstrual.period...Instance.0_Upper.third', 'ts_Blood.clot..DVT..bronchitis..emphysema..asthma..rhinitis..eczema..allergy.diagnosed.by.doctor...Instance.0_5', 'ts_Age.started.smoking.in.former.smokers...Instance.0_Lower.third', 'ts_Time.since.last.prostate.specific.antigen..PSA..test...Instance.0_Upper.third', 'ts_Illness..injury..bereavement..stress.in.last.2.years...Instance.0_1', 'ts_Leisure.social.activities...Instance.0_1', 'ts_Time.employed.in.main.current.job...Instance.0_Lower.third', 'ts_Hip.pain.for.3..months...Instance.0_1', 'ts_Birth.weight.of.first.child...Instance.0_Upper.third', 'ts_Father.s.age...Instance.0_Upper.third', 'ts_Lamb.mutton.intake...Instance.0_2', 'ts_Own.or.rent.accommodation.lived.in...Instance.0_3', 'ts_Snoring...Instance.0_1', 'ts_Time.employed.in.main.current.job...Instance.0_Middle.third', 'ts_Current.tobacco.smoking...Instance.0_0', 'ts_Distance.between.home.and.job.workplace...Instance.0_Lower.third', 'ts_Average.weekly.champagne.plus.white.wine.intake...Instance.0_Upper.third', 'ts_Daytime.dozing...sleeping...Instance.0_2', 'ts_Bread.type...Instance.0_1', 'ts_Reason.for.reducing.amount.of.alcohol.drunk...Instance.0_2', 'ts_Past.tobacco.smoking...Instance.0_4', 'ts_Time.since.last.menstrual.period...Instance.0_Lower.third', 'ts_Age.started.wearing.glasses.or.contact.lenses...Instance.0_Upper.third', 'ts_Length.of.working.week.for.main.job...Instance.0_Lower.third', 'ts_Illnesses.of.siblings...Instance.0_neg27', 'ts_Pain.type.s..experienced.in.last.month...Instance.0_neg7', 'ts_Use.of.sun.uv.protection...Instance.0_3', 'ts_Type.of.tobacco.previously.smoked...Instance.0_3', 'ts_Reason.for.reducing.amount.of.alcohol.drunk...Instance.0_1', 'ts_Age.at.last.live.birth...Instance.0_Upper.third', 'ts_Average.weekly.spirits.intake...Instance.0_Middle.third', 'ts_Able.to.confide...Instance.0_0', 'ts_Length.of.menstrual.cycle...Instance.0_Lower.third', 'ts_Hearing.aid.user...Instance.0_0', 'ts_Illnesses.of.mother...Instance.0_neg17', 'ts_Transport.type.for.commuting.to.job.workplace...Instance.0_3', 'ts_Likelihood.of.resuming.smoking...Instance.0_4', 'ts_Number.of.cigarettes.currently.smoked.daily..current.cigarette.smokers....Instance.0_Middle.third', 'ts_Hair.colour..natural..before.greying....Instance.0_6', 'ts_Snoring...Instance.0_2', 'ts_Pain.type.s..experienced.in.last.month...Instance.0_4', 'ts_Duration.of.other.exercises...Instance.0_3', 'ts_Falls.in.the.last.year...Instance.0_3', 'ts_Age.started.smoking.in.former.smokers...Instance.0_Middle.third', 'ts_Illnesses.of.siblings...Instance.0_8', 'ts_Never.eat.eggs..dairy..wheat..sugar...Instance.0_4', 'ts_Age.stroke.diagnosed...Instance.0_Lower.third', 'ts_Transport.type.for.commuting.to.job.workplace...Instance.0_2', 'ts_Blood.clot..DVT..bronchitis..emphysema..asthma..rhinitis..eczema..allergy.diagnosed.by.doctor...Instance.0_7', 'ts_Why.stopped.smoking...Instance.0_2', 'ts_Neck.shoulder.pain.for.3..months...Instance.0_1', 'ts_Number.of.stillbirths...Instance.0_Middle.third', 'ts_Age.started.smoking.in.current.smokers...Instance.0_Middle.third', 'ts_Illnesses.of.siblings...Instance.0_2', 'ts_Average.weekly.beer.plus.cider.intake...Instance.0_Upper.third', 'ts_Age.asthma.diagnosed...Instance.0_Upper.third', 'ts_Average.total.household.income.before.tax...Instance.0_3', 'ts_Bread.type...Instance.0_3', 'ts_Number.of.cigarettes.previously.smoked.daily...Instance.0_Middle.third', 'ts_Number.of.days.week.walked.10..minutes...Instance.0_neg2', 'ts_Type.of.accommodation.lived.in...Instance.0_4', 'ts_Coffee.type...Instance.0_3', 'ts_Length.of.menstrual.cycle...Instance.0_neg6', 'ts_Types.of.physical.activity.in.last.4.weeks...Instance.0_3', 'ts_Alcohol.intake.versus.10.years.previously...Instance.0_1', 'ts_Time.from.waking.to.first.cigarette...Instance.0_2', 'ts_Difference.in.mobile.phone.use.compared.to.two.years.previously...Instance.0_2', 'ts_Birth.weight.of.first.child...Instance.0_Middle.third', 'ts_Had.menopause...Instance.0_3', 'ts_Answered.sexual.history.questions...Instance.0_1', 'ts_Gestational.diabetes.only...Instance.0_0', 'ts_Most.recent.bowel.cancer.screening...Instance.0_Lower.third', 'ts_Duration.of.other.exercises...Instance.0_4', 'ts_Lamb.mutton.intake...Instance.0_3', 'ts_Number.of.spontaneous.miscarriages...Instance.0_Middle.third', 'ts_Alcohol.drinker.status...Instance.0_1', 'ts_Age.started.smoking.in.current.smokers...Instance.0_Lower.third', 'ts_General.pain.for.3..months...Instance.0_1', 'ts_Father.s.age.at.death...Instance.0_Middle.third', 'ts_Cancer.diagnosed.by.doctor...Instance.0_0', 'ts_Illnesses.of.mother...Instance.0_2', 'ts_Pain.type.s..experienced.in.last.month...Instance.0_3', 'ts_Duration.walking.for.pleasure...Instance.0_3', 'ts_Transport.type.for.commuting.to.job.workplace...Instance.0_4', 'ts_Frequency.of.stair.climbing.in.last.4.weeks...Instance.0_0', 'ts_Ever.had.breast.cancer.screening...mammogram...Instance.0_1', 'ts_Age.emphysema.chronic.bronchitis.diagnosed...Instance.0_Middle.third', 'ts_Exposure.to.tobacco.smoke.outside.home...Instance.0_Middle.third', 'ts_Sleeplessness...insomnia...Instance.0_3', 'ts_Age.hay.fever..rhinitis.or.eczema.diagnosed...Instance.0_Lower.third', 'ts_Age.completed.full.time.education...Instance.0_Upper.third', 'ts_Own.or.rent.accommodation.lived.in...Instance.0_1', 'ts_Past.tobacco.smoking...Instance.0_3', 'ts_Job.involves.mainly.walking.or.standing...Instance.0_2', 'ts_Hair.colour..natural..before.greying....Instance.0_4', 'ts_Frequency.of.tiredness...lethargy.in.last.2.weeks...Instance.0_4', 'ts_Age.pulmonary.embolism..blood.clot.in.lung..diagnosed...Instance.0_Upper.third', 'ts_Why.stopped.smoking...Instance.0_1', 'ts_Age.at.last.live.birth...Instance.0_Middle.third', 'ts_Frequency.of.travelling.from.home.to.job.workplace...Instance.0_Lower.third', 'ts_Combined.sex.medication_5', 'ts_Seen.a.psychiatrist.for.nerves..anxiety..tension.or.depression...Instance.0_1', 'ts_Illnesses.of.siblings...Instance.0_9', 'ts_Illnesses.of.siblings...Instance.0_3', 'ts_Duration.of.vigorous.activity...Instance.0_Middle.third', 'ts_Alcohol.drinker.status...Instance.0_2', 'ts_Age.emphysema.chronic.bronchitis.diagnosed...Instance.0_Upper.third', 'ts_Answered.sexual.history.questions...Instance.0_2', 'ts_Current.employment.status...Instance.0_3', 'ts_Frequency.of.walking.for.pleasure.in.last.4.weeks...Instance.0_2', 'ts_Ever.had.stillbirth..spontaneous.miscarriage.or.termination...Instance.0_1', 'ts_Time.from.waking.to.first.cigarette...Instance.0_1', 'ts_Time.employed.in.main.current.job...Instance.0_Upper.third', 'ts_Ethnic.background...Instance.0_3', 'ts_Processed.meat.intake...Instance.0_3', 'ts_Pain.type.s..experienced.in.last.month...Instance.0_8', 'ts_Cancer.diagnosed.by.doctor...Instance.0_1', 'ts_Smoking.compared.to.10.years.previous...Instance.0_2', 'ts_Falls.in.the.last.year...Instance.0_1', 'ts_Illnesses.of.siblings...Instance.0_4', 'ts_Smoking.compared.to.10.years.previous...Instance.0_3', 'ts_Years.since.last.cervical.smear.test...Instance.0_Middle.third', 'ts_Frequency.of.other.exercises.in.last.4.weeks...Instance.0_4', 'ts_Light.smokers..at.least.100.smokes.in.lifetime...Instance.0_0', 'ts_Usual.walking.pace...Instance.0_2', 'ts_Reason.former.drinker.stopped.drinking.alcohol...Instance.0_1', 'ts_Smoking.status...Instance.0_1', 'ts_Ease.of.skin.tanning...Instance.0_1', 'ts_Pork.intake...Instance.0_2', 'ts_Why.reduced.smoking...Instance.0_2', 'ts_Alcohol.intake.frequency....Instance.0_6', 'ts_Wants.to.stop.smoking...Instance.0_3', 'ts_Seen.a.psychiatrist.for.nerves..anxiety..tension.or.depression...Instance.0_0', 'ts_Cereal.type...Instance.0_4', 'ts_Age.at.first.live.birth...Instance.0_Middle.third', 'ts_Salt.added.to.food...Instance.0_1', 'ts_Usual.side.of.head.for.mobile.phone.use...Instance.0_2', 'ts_Getting.up.in.morning...Instance.0_3', 'ts_Age.hay.fever..rhinitis.or.eczema.diagnosed...Instance.0_Middle.third', 'ts_Mouth.teeth.dental.problems...Instance.0_4', 'ts_Average.total.household.income.before.tax...Instance.0_5', 'ts_Years.since.last.breast.cancer.screening...mammogram...Instance.0_Lower.third', 'ts_Type.of.tobacco.currently.smoked...Instance.0_1', 'ts_Medication.for.pain.relief..constipation..heartburn...Instance.0_4', 'ts_Duration.of.strenuous.sports...Instance.0_3', 'ts_Type.of.tobacco.currently.smoked...Instance.0_3', 'ts_Number.of.vehicles.in.household...Instance.0_1', 'ts_Difference.in.mobile.phone.use.compared.to.two.years.previously...Instance.0_1', 'ts_Getting.up.in.morning...Instance.0_4', 'ts_Hair.balding.pattern...Instance.0_1', 'ts_Type.of.tobacco.currently.smoked...Instance.0_2', 'ts_Processed.meat.intake...Instance.0_1', 'ts_Number.of.pregnancy.terminations...Instance.0_Middle.third', 'ts_Why.stopped.smoking...Instance.0_4', 'ts_Blood.clot..DVT..bronchitis..emphysema..asthma..rhinitis..eczema..allergy.diagnosed.by.doctor...Instance.0_9', 'ts_Reason.former.drinker.stopped.drinking.alcohol...Instance.0_2', 'ts_Age.of.primiparous.women.at.birth.of.child...Instance.0_Upper.third', 'ts_Length.of.mobile.phone.use...Instance.0_3', 'ts_Current.employment.status...Instance.0_7', 'ts_Gas.or.solid.fuel.cooking.heating...Instance.0_1', 'ts_Age.of.primiparous.women.at.birth.of.child...Instance.0_Middle.third', 'ts_Frequency.of.stair.climbing.in.last.4.weeks...Instance.0_3', 'ts_Alcohol.usually.taken.with.meals...Instance.0_0', 'ts_Frequency.of.light.DIY.in.last.4.weeks...Instance.0_3', 'ts_Salt.added.to.food...Instance.0_4', 'ts_Facial.ageing...Instance.0_1', 'ts_Age.stopped.smoking...Instance.0_Middle.third', 'ts_Non.butter.spread.type.details...Instance.0_4', 'ts_Gas.or.solid.fuel.cooking.heating...Instance.0_3', 'ts_Spread.type...Instance.0_0', 'ts_Wants.to.stop.smoking...Instance.0_1', 'ts_Salad...raw.vegetable.intake...Instance.0', 'ts_Lamb.mutton.intake...Instance.0_1', 'ts_Gestational.diabetes.only...Instance.0_neg2', 'ts_Frequency.of.other.exercises.in.last.4.weeks...Instance.0_3', 'ts_Number.of.unsuccessful.stop.smoking.attempts...Instance.0_Middle.third', 'ts_Facial.ageing...Instance.0_2', 'ts_Distance.between.home.and.job.workplace...Instance.0_Upper.third', 'ts_Number.of.unsuccessful.stop.smoking.attempts...Instance.0_Upper.third', 'ts_Previously.smoked.cigarettes.on.most.all.days...Instance.0_1', 'ts_Illnesses.of.siblings...Instance.0_6', 'ts_Length.of.mobile.phone.use...Instance.0_0', 'ts_Frequency.of.strenuous.sports.in.last.4.weeks...Instance.0_4', 'ts_Relative.age.of.first.facial.hair...Instance.0_1', 'ts_Alcohol.usually.taken.with.meals...Instance.0_1', 'ts_Age.when.periods.started..menarche....Instance.0_Upper.third', 'ts_Number.in.household...Instance.0_Middle.third', 'ts_Duration.walking.for.pleasure...Instance.0_4', 'ts_Beef.intake...Instance.0_3', 'ts_Qualifications...Instance.0_6', 'ts_Average.weekly.spirits.intake...Instance.0_Upper.third', 'ts_Frequency.of.friend.family.visits...Instance.0_6', 'ts_Why.stopped.smoking...Instance.0_3', 'ts_Reason.for.reducing.amount.of.alcohol.drunk...Instance.0_4', 'ts_Leisure.social.activities...Instance.0_2', 'ts_Number.of.full.sisters...Instance.0_Middle.third', 'ts_Frequency.of.walking.for.pleasure.in.last.4.weeks...Instance.0_1', 'ts_Smoking.smokers.in.household...Instance.0_0', 'ts_Age.first.had.sexual.intercourse...Instance.0_Middle.third', 'ts_Types.of.transport.used..excluding.work....Instance.0_2', 'ts_Types.of.physical.activity.in.last.4.weeks...Instance.0_4', 'ts_Most.recent.bowel.cancer.screening...Instance.0_Upper.third', 'ts_Illnesses.of.siblings...Instance.0_13', 'ts_Illnesses.of.father...Instance.0_2', 'ts_Duration.of.moderate.activity...Instance.0_Middle.third', 'ts_Pork.intake...Instance.0_3', 'ts_Number.of.vehicles.in.household...Instance.0_3', 'ts_Number.of.unsuccessful.stop.smoking.attempts...Instance.0_Lower.third', 'ts_Age.emphysema.chronic.bronchitis.diagnosed...Instance.0_Lower.third', 'ts_Ease.of.skin.tanning...Instance.0_3', 'ts_Weight.change.compared.with.1.year.ago...Instance.0_3', 'ts_Frequency.of.stair.climbing.in.last.4.weeks...Instance.0_1', 'ts_Stomach.abdominal.pain.for.3..months...Instance.0_1', 'ts_Gas.or.solid.fuel.cooking.heating...Instance.0_neg7', 'ts_Number.in.household...Instance.0_Lower.third', 'ts_Water.intake...Instance.0', 'ts_Frequency.of.other.exercises.in.last.4.weeks...Instance.0_2', 'ts_Duration.of.strenuous.sports...Instance.0_4', 'ts_Types.of.transport.used..excluding.work....Instance.0_3', 'ts_Time.from.waking.to.first.cigarette...Instance.0_3', 'ts_Sleeplessness...insomnia...Instance.0_1', 'ts_Number.of.pregnancy.terminations...Instance.0_Upper.third', 'ts_Number.of.cigarettes.previously.smoked.daily..current.cigar.pipe.smokers....Instance.0_Middle.third', 'ts_Non.butter.spread.type.details...Instance.0_2', 'ts_Never.eat.eggs..dairy..wheat..sugar...Instance.0_3', 'ts_Age.when.periods.started..menarche....Instance.0_Lower.third', 'ts_Frequency.of.unenthusiasm...disinterest.in.last.2.weeks...Instance.0_1', 'ts_Beef.intake...Instance.0_1', 'ts_Use.of.sun.uv.protection...Instance.0_2', 'ts_Frequency.of.light.DIY.in.last.4.weeks...Instance.0_2', 'ts_Number.of.full.sisters...Instance.0_Upper.third', 'ts_Frequency.of.friend.family.visits...Instance.0_1', 'ts_Mouth.teeth.dental.problems...Instance.0_3', 'ts_Tea.intake...Instance.0', 'ts_Why.reduced.smoking...Instance.0_1', 'ts_Hair.colour..natural..before.greying....Instance.0_5', 'ts_Salt.added.to.food...Instance.0_3', 'ts_Frequency.of.heavy.DIY.in.last.4.weeks...Instance.0_5', 'ts_Age.started.smoking.in.former.smokers...Instance.0_Upper.third', 'ts_Frequency.of.stair.climbing.in.last.4.weeks...Instance.0_4', 'ts_Illnesses.of.mother...Instance.0_neg27', 'ts_Wants.to.stop.smoking...Instance.0_4', 'ts_Frequency.of.heavy.DIY.in.last.4.weeks...Instance.0_4', 'ts_Frequency.of.strenuous.sports.in.last.4.weeks...Instance.0_3', 'ts_Wants.to.stop.smoking...Instance.0_2', 'ts_Illnesses.of.mother...Instance.0_6', 'ts_Age.deep.vein.thrombosis..DVT..blood.clot.in.leg..diagnosed...Instance.0_Middle.third', 'ts_Alcohol.intake.versus.10.years.previously...Instance.0_3', 'ts_Poultry.intake...Instance.0_2', 'ts_Cheese.intake...Instance.0_2', 'ts_Cereal.type...Instance.0_5', 'ts_Frequency.of.friend.family.visits...Instance.0_7', 'ts_Had.menopause...Instance.0_1', 'ts_Illnesses.of.siblings...Instance.0_10', 'ts_Frequency.of.tenseness...restlessness.in.last.2.weeks...Instance.0_3', 'ts_Illnesses.of.father...Instance.0_6', 'ts_Medication.for.pain.relief..constipation..heartburn...Instance.0_6', 'ts_Length.of.working.week.for.main.job...Instance.0_Upper.third', 'ts_Difficulty.not.smoking.for.1.day...Instance.0_3', 'ts_Mother.s.age...Instance.0_Upper.third', 'ts_Able.to.confide...Instance.0_4', 'ts_Frequency.of.friend.family.visits...Instance.0_4', 'ts_Ethnic.background...Instance.0_3002', 'ts_Duration.of.walks...Instance.0_Middle.third', 'ts_Types.of.transport.used..excluding.work....Instance.0_4', 'ts_Frequency.of.walking.for.pleasure.in.last.4.weeks...Instance.0_3', 'ts_Ever.tried.to.stop.smoking...Instance.0_2', 'ts_Frequency.of.tenseness...restlessness.in.last.2.weeks...Instance.0_4', 'ts_Years.since.last.breast.cancer.screening...mammogram...Instance.0_Middle.third', 'ts_Beef.intake...Instance.0_2', 'ts_Why.reduced.smoking...Instance.0_4', 'ts_Never.eat.eggs..dairy..wheat..sugar...Instance.0_2', 'ts_Relative.age.voice.broke...Instance.0_1', 'ts_Smoking.compared.to.10.years.previous...Instance.0_1', 'ts_Number.of.vehicles.in.household...Instance.0_4', 'ts_Job.involves.heavy.manual.or.physical.work...Instance.0_2', 'ts_Year.immigrated.to.UK..United.Kingdom....Instance.0_Upper.third', 'ts_Duration.of.other.exercises...Instance.0_2', 'ts_Lamb.mutton.intake...Instance.0_0', 'ts_Facial.ageing...Instance.0_3', 'ts_Age.stopped.smoking.cigarettes..current.cigar.pipe.or.previous.cigarette.smoker....Instance.0_Middle.third', 'ts_Alcohol.intake.frequency....Instance.0_2', 'ts_Coffee.type...Instance.0_2', 'ts_Skin.colour...Instance.0_3', 'ts_Age.pulmonary.embolism..blood.clot.in.lung..diagnosed...Instance.0_Middle.third', 'ts_Number.of.full.brothers...Instance.0_Middle.third', 'ts_Frequency.of.other.exercises.in.last.4.weeks...Instance.0_5', 'ts_Frequency.of.depressed.mood.in.last.2.weeks...Instance.0_3', 'ts_Spread.type...Instance.0_3', 'ts_Duration.of.light.DIY...Instance.0_3', 'ts_Illnesses.of.siblings...Instance.0_11', 'ts_Illness..injury..bereavement..stress.in.last.2.years...Instance.0_2', 'ts_Age.stopped.smoking.cigarettes..current.cigar.pipe.or.previous.cigarette.smoker....Instance.0_Upper.third', 'ts_Blood.clot..DVT..bronchitis..emphysema..asthma..rhinitis..eczema..allergy.diagnosed.by.doctor...Instance.0_8', 'ts_Why.stopped.smoking...Instance.0_neg7', 'ts_Cheese.intake...Instance.0_0', 'ts_Age.last.used.hormone.replacement.therapy..HRT....Instance.0_neg11', 'ts_Cheese.intake...Instance.0_5', 'ts_Pain.type.s..experienced.in.last.month...Instance.0_5', 'ts_Cereal.type...Instance.0_2', 'ts_Average.total.household.income.before.tax...Instance.0_2', 'ts_Number.of.vehicles.in.household...Instance.0_2', 'ts_Hair.balding.pattern...Instance.0_2', 'ts_Number.of.full.brothers...Instance.0_Upper.third', 'ts_Duration.of.moderate.activity...Instance.0_Upper.third', 'ts_Maternal.smoking.around.birth...Instance.0_1', 'ts_Ethnic.background...Instance.0_1003', 'ts_Average.monthly.beer.plus.cider.intake...Instance.0_Upper.third', 'ts_Alcohol.intake.versus.10.years.previously...Instance.0_2', 'ts_Frequency.of.strenuous.sports.in.last.4.weeks...Instance.0_5', 'ts_Frequency.of.friend.family.visits...Instance.0_3', 'ts_Frequency.of.strenuous.sports.in.last.4.weeks...Instance.0_2', 'ts_Beef.intake...Instance.0_0', 'ts_Fractured.broken.bones.in.last.5.years...Instance.0_0', 'ts_Ethnic.background...Instance.0_5', 'ts_How.are.people.in.household.related.to.participant...Instance.0_6', 'ts_Milk.type.used...Instance.0_4', 'ts_Worry.too.long.after.embarrassment...Instance.0_0', 'ts_Frequency.of.stair.climbing.in.last.4.weeks...Instance.0_5', 'ts_Likelihood.of.resuming.smoking...Instance.0_2', 'ts_Bread.type...Instance.0_2', 'ts_Loneliness..isolation...Instance.0_1', 'ts_Birth.weight.of.first.child...Instance.0_Lower.third', 'ts_Use.of.sun.uv.protection...Instance.0_4', 'ts_Suffer.from..nerves....Instance.0_0', 'ts_Variation.in.diet...Instance.0_3', 'ts_Cheese.intake...Instance.0_4', 'ts_Illnesses.of.mother...Instance.0_3', 'ts_Illnesses.of.siblings...Instance.0_5', 'ts_Ethnic.background...Instance.0_3001', 'ts_Ever.tried.to.stop.smoking...Instance.0_0', 'ts_Gas.or.solid.fuel.cooking.heating...Instance.0_2', 'ts_Comparative.body.size.at.age.10...Instance.0_1', 'ts_Why.reduced.smoking...Instance.0_3', 'ts_Non.butter.spread.type.details...Instance.0_5', 'ts_Frequency.of.unenthusiasm...disinterest.in.last.2.weeks...Instance.0_3', 'ts_Frequency.of.unenthusiasm...disinterest.in.last.2.weeks...Instance.0_4', 'ts_Never.eat.eggs..dairy..wheat..sugar...Instance.0_1', 'ts_Duration.of.strenuous.sports...Instance.0_2', 'ts_Fracture.resulting.from.simple.fall...Instance.0_1', 'ts_Worry.too.long.after.embarrassment...Instance.0_1', 'ts_Hip.pain.for.3..months...Instance.0_0', 'ts_Mouth.teeth.dental.problems...Instance.0_2', 'ts_Age.pulmonary.embolism..blood.clot.in.lung..diagnosed...Instance.0_Lower.third', 'ts_Combined.sex.medication_4', 'ts_Duration.of.vigorous.activity...Instance.0_Upper.third', 'ts_Father.s.age.at.death...Instance.0_Upper.third', 'ts_Likelihood.of.resuming.smoking...Instance.0_3', 'ts_Years.since.last.breast.cancer.screening...mammogram...Instance.0_Upper.third', 'ts_Frequency.of.tiredness...lethargy.in.last.2.weeks...Instance.0_1', 'ts_Risk.taking...Instance.0_0', 'ts_Processed.meat.intake...Instance.0_4', 'ts_Pork.intake...Instance.0_1', 'ts_Illness..injury..bereavement..stress.in.last.2.years...Instance.0_4', 'ts_Fresh.fruit.intake...Instance.0', 'ts_Loneliness..isolation...Instance.0_0', 'ts_Type.of.accommodation.lived.in...Instance.0_1', 'ts_Illnesses.of.father...Instance.0_neg27', 'ts_Length.of.mobile.phone.use...Instance.0_1', 'ts_Exposure.to.tobacco.smoke.at.home...Instance.0_Upper.third', 'ts_Job.involves.mainly.walking.or.standing...Instance.0_4', 'ts_Chest.pain.or.discomfort.when.walking.uphill.or.hurrying...Instance.0_0', 'ts_Sleeplessness...insomnia...Instance.0_2', 'ts_Age.started.smoking.in.current.smokers...Instance.0_Upper.third', 'ts_Illnesses.of.mother...Instance.0_8', 'ts_Coffee.intake...Instance.0', 'ts_Fed.up.feelings...Instance.0_0', 'ts_Frequency.of.other.exercises.in.last.4.weeks...Instance.0_1', 'ts_Vitamin.and.mineral.supplements...Instance.0_6', 'ts_Suffer.from..nerves....Instance.0_1', 'ts_Risk.taking...Instance.0_1', 'ts_Former.alcohol.drinker...Instance.0_0', 'ts_Frequency.of.solarium.sunlamp.use...Instance.0', 'ts_Processed.meat.intake...Instance.0_5', 'ts_Fractured.broken.bones.in.last.5.years...Instance.0_1', 'ts_Handedness..chirality.laterality....Instance.0_3', 'ts_Mineral.and.other.dietary.supplements...Instance.0_5', 'ts_Fed.up.feelings...Instance.0_1', 'ts_Reason.former.drinker.stopped.drinking.alcohol...Instance.0_3', 'ts_Frequency.of.tiredness...lethargy.in.last.2.weeks...Instance.0_3', 'ts_Number.of.vehicles.in.household...Instance.0_5', 'ts_Age.deep.vein.thrombosis..DVT..blood.clot.in.leg..diagnosed...Instance.0_Lower.third', 'ts_Morning.evening.person..chronotype....Instance.0_2', 'ts_Age.at.menopause..last.menstrual.period....Instance.0_Upper.third', 'ts_Ethnic.background...Instance.0_1002', 'ts_Average.weekly.fortified.wine.intake...Instance.0_Middle.third', 'ts_Illnesses.of.mother...Instance.0_5', 'ts_Illnesses.of.father...Instance.0_13', 'ts_Duration.of.strenuous.sports...Instance.0_5', 'ts_Number.of.days.week.of.vigorous.physical.activity.10..minutes...Instance.0', 'ts_Alcohol.intake.frequency....Instance.0_5', 'ts_Reason.former.drinker.stopped.drinking.alcohol...Instance.0_4', 'ts_Spread.type...Instance.0_2', 'ts_Oily.fish.intake...Instance.0_1', 'ts_Frequency.of.unenthusiasm...disinterest.in.last.2.weeks...Instance.0_2', 'ts_Alcohol.usually.taken.with.meals...Instance.0_neg6', 'ts_Duration.of.other.exercises...Instance.0_7', 'ts_Age.last.used.hormone.replacement.therapy..HRT....Instance.0_Upper.third', 'ts_Milk.type.used...Instance.0_1', 'ts_Falls.in.the.last.year...Instance.0_2', 'ts_Hands.free.device.speakerphone.use.with.mobile.phone.in.last.3.month...Instance.0_0', 'ts_Mineral.and.other.dietary.supplements...Instance.0_4', 'ts_Poultry.intake...Instance.0_3', 'ts_Use.of.sun.uv.protection...Instance.0_5', 'ts_Job.involves.night.shift.work...Instance.0_1', 'ts_Pork.intake...Instance.0_0', 'ts_Birth.weight.of.first.child...Instance.0_neg2', 'ts_Previously.smoked.cigarettes.on.most.all.days...Instance.0_0', 'ts_Types.of.transport.used..excluding.work....Instance.0_1', 'ts_Weekly.usage.of.mobile.phone.in.last.3.months...Instance.0_2', 'ts_Number.of.children.fathered...Instance.0_Lower.third', 'ts_Illnesses.of.adopted.siblings...Instance.0_2', 'ts_Number.of.cigarettes.previously.smoked.daily..current.cigar.pipe.smokers....Instance.0_Lower.third', 'ts_Weekly.usage.of.mobile.phone.in.last.3.months...Instance.0_3', 'ts_Duration.of.heavy.DIY...Instance.0_7', 'ts_Skin.colour...Instance.0_2', 'ts_Frequency.of.heavy.DIY.in.last.4.weeks...Instance.0_1', 'ts_Illnesses.of.mother...Instance.0_4', 'ts_Illnesses.of.father...Instance.0_10', 'ts_Fractured.bone.site.s....Instance.0_3', 'ts_Past.tobacco.smoking...Instance.0_2', 'ts_Morning.evening.person..chronotype....Instance.0_1', 'ts_Ethnic.background...Instance.0_1', 'ts_Medication.for.pain.relief..constipation..heartburn...Instance.0_3', 'ts_Oily.fish.intake...Instance.0_3', 'ts_Illnesses.of.father...Instance.0_3', 'ts_Illnesses.of.adopted.siblings...Instance.0_9', 'ts_Comparative.body.size.at.age.10...Instance.0_3', 'ts_Able.to.confide...Instance.0_5', 'ts_Non.butter.spread.type.details...Instance.0_6', 'ts_Oily.fish.intake...Instance.0_5', 'ts_Age.started.wearing.glasses.or.contact.lenses...Instance.0_Lower.third', 'ts_Number.of.cigarettes.previously.smoked.daily..current.cigar.pipe.smokers....Instance.0_Upper.third', 'ts_Frequency.of.heavy.DIY.in.last.4.weeks...Instance.0_6', 'ts_Reason.for.reducing.amount.of.alcohol.drunk...Instance.0_3', 'ts_Frequency.of.strenuous.sports.in.last.4.weeks...Instance.0_1', 'ts_Mood.swings...Instance.0_0', 'ts_Getting.up.in.morning...Instance.0_2', 'ts_Beef.intake...Instance.0_4', 'ts_Age.started.oral.contraceptive.pill...Instance.0_Upper.third', 'ts_Ethnic.background...Instance.0_2003', 'ts_Weekly.usage.of.mobile.phone.in.last.3.months...Instance.0_0', 'ts_Own.or.rent.accommodation.lived.in...Instance.0_5', 'ts_Mood.swings...Instance.0_1', 'ts_Why.reduced.smoking...Instance.0_neg7', 'ts_Illness..injury..bereavement..stress.in.last.2.years...Instance.0_5', 'ts_Average.weekly.red.wine.intake...Instance.0_Middle.third', 'ts_Fractured.bone.site.s....Instance.0_5', 'ts_Duration.of.light.DIY...Instance.0_7', 'ts_Drive.faster.than.motorway.speed.limit...Instance.0_4', 'ts_Illnesses.of.father...Instance.0_8', 'ts_Type.of.accommodation.lived.in...Instance.0_2', 'ts_Fractured.bone.site.s....Instance.0_7', 'ts_Usual.walking.pace...Instance.0_neg7', 'ts_Age.first.had.sexual.intercourse...Instance.0_Lower.third', 'ts_Frequency.of.tenseness...restlessness.in.last.2.weeks...Instance.0_1', 'ts_Able.to.confide...Instance.0_3', 'ts_Job.involves.shift.work...Instance.0_2', 'ts_Cooked.vegetable.intake...Instance.0', 'ts_How.are.people.in.household.related.to.participant...Instance.0_8', 'ts_Weight.change.compared.with.1.year.ago...Instance.0_0', 'ts_Reason.former.drinker.stopped.drinking.alcohol...Instance.0_5', 'ts_Difference.in.mobile.phone.use.compared.to.two.years.previously...Instance.0_3', 'ts_Number.of.live.births...Instance.0_Upper.third', 'ts_Duration.walking.for.pleasure...Instance.0_1', 'ts_Job.involves.shift.work...Instance.0_4', 'ts_Vitamin.and.mineral.supplements...Instance.0_1', 'ts_Illnesses.of.adopted.mother...Instance.0_2', 'ts_Job.involves.shift.work...Instance.0_3', 'ts_Hot.drink.temperature...Instance.0_1', 'ts_Ethnic.background...Instance.0_4002', 'ts_Type.of.accommodation.lived.in...Instance.0_neg7', 'ts_Ever.had.hysterectomy..womb.removed....Instance.0_1', 'ts_Average.weekly.red.wine.intake...Instance.0_Upper.third', 'ts_Type.of.accommodation.lived.in...Instance.0_5', 'ts_Adopted.mother.still.alive...Instance.0_0', 'ts_Mouth.teeth.dental.problems...Instance.0_5', 'ts_Duration.of.walks...Instance.0_Upper.third', 'ts_Illnesses.of.mother...Instance.0_10', 'ts_Fractured.bone.site.s....Instance.0_6', 'ts_Duration.of.light.DIY...Instance.0_4', 'ts_Mineral.and.other.dietary.supplements...Instance.0_3', 'ts_Cereal.type...Instance.0_1', 'ts_Hands.free.device.speakerphone.use.with.mobile.phone.in.last.3.month...Instance.0_1', 'ts_Getting.up.in.morning...Instance.0_1', 'ts_Mineral.and.other.dietary.supplements...Instance.0_1', 'ts_Seen.doctor..GP..for.nerves..anxiety..tension.or.depression...Instance.0_1', 'ts_Illnesses.of.mother...Instance.0_9', 'ts_Non.butter.spread.type.details...Instance.0_9', 'ts_Exposure.to.tobacco.smoke.outside.home...Instance.0_Upper.third', 'ts_How.are.people.in.household.related.to.participant...Instance.0_1', 'ts_Irritability...Instance.0_0', 'ts_Vitamin.and.mineral.supplements...Instance.0_neg7', 'ts_Reason.for.reducing.amount.of.alcohol.drunk...Instance.0_5', 'ts_Illness..injury..bereavement..stress.in.last.2.years...Instance.0_neg7', 'ts_Seen.doctor..GP..for.nerves..anxiety..tension.or.depression...Instance.0_0', 'ts_Ease.of.skin.tanning...Instance.0_2', 'ts_Illnesses.of.father...Instance.0_9', 'ts_Frequency.of.light.DIY.in.last.4.weeks...Instance.0_6', 'ts_Weekly.usage.of.mobile.phone.in.last.3.months...Instance.0_5', 'ts_Skin.colour...Instance.0_6', 'ts_Job.involves.night.shift.work...Instance.0_2', 'ts_Frequency.of.stair.climbing.in.last.4.weeks...Instance.0_2', 'ts_Headaches.for.3..months...Instance.0_0', 'ts_Duration.of.other.exercises...Instance.0_5', 'ts_Tense....highly.strung....Instance.0_0', 'ts_Relative.age.of.first.facial.hair...Instance.0_3', 'ts_Frequency.of.heavy.DIY.in.last.4.weeks...Instance.0_3', 'ts_Cereal.type...Instance.0_3', 'ts_Age.completed.full.time.education...Instance.0_neg2', 'ts_Duration.of.strenuous.sports...Instance.0_6', 'ts_Fractured.bone.site.s....Instance.0_2', 'ts_Types.of.transport.used..excluding.work....Instance.0_neg7', 'ts_Lamb.mutton.intake...Instance.0_5', 'ts_Non.oily.fish.intake...Instance.0_2', 'ts_Frequency.of.depressed.mood.in.last.2.weeks...Instance.0_1', 'ts_Number.of.spontaneous.miscarriages...Instance.0_Upper.third', 'ts_Ethnic.background...Instance.0_6', 'ts_Average.monthly.intake.of.other.alcoholic.drinks...Instance.0_Middle.third', 'ts_Frequency.of.light.DIY.in.last.4.weeks...Instance.0_1', 'ts_Drive.faster.than.motorway.speed.limit...Instance.0_3', 'ts_Adopted.father.still.alive...Instance.0_0', 'ts_Irritability...Instance.0_1', 'ts_How.are.people.in.household.related.to.participant...Instance.0_5', 'ts_Ethnic.background...Instance.0_4001', 'ts_Tense....highly.strung....Instance.0_1', 'ts_Average.monthly.fortified.wine.intake...Instance.0_Upper.third', 'ts_Weekly.usage.of.mobile.phone.in.last.3.months...Instance.0_1', 'ts_Comparative.height.size.at.age.10...Instance.0_2', 'ts_Transport.type.for.commuting.to.job.workplace...Instance.0_neg7', 'ts_Illnesses.of.father...Instance.0_11', 'ts_Major.dietary.changes.in.the.last.5.years...Instance.0_2', 'ts_Age.stopped.smoking.cigarettes..current.cigar.pipe.or.previous.cigarette.smoker....Instance.0_Lower.third', 'ts_Time.spent.driving...Instance.0', 'ts_Type.of.accommodation.lived.in...Instance.0_3', 'ts_Job.involves.night.shift.work...Instance.0_3', 'ts_Length.of.mobile.phone.use...Instance.0_4', 'ts_Ethnic.background...Instance.0_2002', 'ts_Illnesses.of.adopted.siblings...Instance.0_8', 'ts_Leisure.social.activities...Instance.0_4', 'ts_Average.weekly.champagne.plus.white.wine.intake...Instance.0_Middle.third', 'ts_Duration.walking.for.pleasure...Instance.0_6', 'ts_Oily.fish.intake...Instance.0_0', 'ts_Illnesses.of.adopted.mother...Instance.0_4', 'ts_Salt.added.to.food...Instance.0_2', 'ts_Illness..injury..bereavement..stress.in.last.2.years...Instance.0_6', 'ts_General.pain.for.3..months...Instance.0_0', 'ts_Leisure.social.activities...Instance.0_neg7', 'ts_Weekly.usage.of.mobile.phone.in.last.3.months...Instance.0_4', 'ts_Pain.type.s..experienced.in.last.month...Instance.0_1', 'ts_Time.from.waking.to.first.cigarette...Instance.0_4', 'ts_Dried.fruit.intake...Instance.0', 'ts_Frequency.of.friend.family.visits...Instance.0_2', 'ts_Age.at.hysterectomy...Instance.0_Lower.third', 'ts_Type.of.tobacco.previously.smoked...Instance.0_neg7', 'ts_Age.last.used.hormone.replacement.therapy..HRT....Instance.0_Middle.third', 'ts_Likelihood.of.resuming.smoking...Instance.0_1', 'ts_Current.employment.status...Instance.0_neg7', 'ts_Ethnic.background...Instance.0_4003', 'ts_Number.of.cigarettes.previously.smoked.daily...Instance.0_Lower.third', 'ts_Duration.walking.for.pleasure...Instance.0_5', 'ts_Drive.faster.than.motorway.speed.limit...Instance.0_1', 'ts_Illnesses.of.adopted.siblings...Instance.0_11', 'ts_Lifetime.number.of.sexual.partners...Instance.0_Lower.third', 'ts_Duration.of.heavy.DIY...Instance.0_4', 'ts_Average.weekly.fortified.wine.intake...Instance.0_Upper.third', 'ts_Facial.pains.for.3..months...Instance.0_1', 'ts_Hands.free.device.speakerphone.use.with.mobile.phone.in.last.3.month...Instance.0_2', 'ts_Non.oily.fish.intake...Instance.0_1', 'ts_Lifetime.number.of.sexual.partners...Instance.0_Upper.third', 'ts_Plays.computer.games...Instance.0_0', 'ts_Spread.type...Instance.0_1', 'ts_Fractured.bone.site.s....Instance.0_1', 'ts_Poultry.intake...Instance.0_4', 'ts_Milk.type.used...Instance.0_3', 'ts_Duration.of.strenuous.sports...Instance.0_1', 'ts_Part.of.a.multiple.birth...Instance.0_1', 'ts_How.are.people.in.household.related.to.participant...Instance.0_4', 'ts_Drive.faster.than.motorway.speed.limit...Instance.0_5', 'ts_Age.hay.fever..rhinitis.or.eczema.diagnosed...Instance.0_Upper.third', 'ts_Usual.side.of.head.for.mobile.phone.use...Instance.0_1', 'ts_Fractured.bone.site.s....Instance.0_4', 'ts_Skin.colour...Instance.0_5', 'ts_Milk.type.used...Instance.0_6', 'ts_Non.oily.fish.intake...Instance.0_3', 'ts_Cereal.intake...Instance.0', 'ts_Variation.in.diet...Instance.0_2', 'ts_Ethnic.background...Instance.0_4', 'ts_Alcohol.intake.frequency....Instance.0_1', 'ts_Age.at.hysterectomy...Instance.0_Middle.third', 'ts_Able.to.confide...Instance.0_2', 'ts_Age.completed.full.time.education...Instance.0_Middle.third', 'ts_Length.of.mobile.phone.use...Instance.0_2', 'ts_Part.of.a.multiple.birth...Instance.0_0', 'ts_Age.asthma.diagnosed...Instance.0_Lower.third', 'ts_Duration.of.heavy.DIY...Instance.0_5', 'ts_Sensitivity...hurt.feelings...Instance.0_1', 'ts_Duration.of.vigorous.activity...Instance.0_Lower.third', 'ts_Duration.of.light.DIY...Instance.0_6', 'ts_Job.involves.night.shift.work...Instance.0_4', 'ts_Bread.type...Instance.0_4', 'ts_Number.of.days.week.walked.10..minutes...Instance.0_Middle.third', 'ts_Average.monthly.fortified.wine.intake...Instance.0_Middle.third', 'ts_Relative.age.voice.broke...Instance.0_3', 'ts_Smoking.smokers.in.household...Instance.0_1', 'ts_Current.employment.status...Instance.0_5', 'ts_Type.of.tobacco.currently.smoked...Instance.0_neg7', 'ts_Frequency.of.walking.for.pleasure.in.last.4.weeks...Instance.0_6', 'ts_Vitamin.and.mineral.supplements...Instance.0_3', 'ts_Frequency.of.walking.for.pleasure.in.last.4.weeks...Instance.0_4', 'ts_Stomach.abdominal.pain.for.3..months...Instance.0_0', 'ts_Job.involves.heavy.manual.or.physical.work...Instance.0_3', 'ts_Pain.type.s..experienced.in.last.month...Instance.0_2', 'ts_Smoking.smokers.in.household...Instance.0_2', 'ts_Adopted.father.still.alive...Instance.0_1', 'ts_Mineral.and.other.dietary.supplements...Instance.0_6', 'ts_Non.oily.fish.intake...Instance.0_4', 'ts_Number.of.stillbirths...Instance.0_Upper.third', 'ts_Comparative.height.size.at.age.10...Instance.0_3', 'ts_Beef.intake...Instance.0_5', 'ts_Age.first.had.sexual.intercourse...Instance.0_Upper.third', 'ts_Handedness..chirality.laterality....Instance.0_2', 'ts_Age.started.hormone.replacement.therapy..HRT....Instance.0_Middle.third', 'ts_Cheese.intake...Instance.0_1', 'ts_Able.to.confide...Instance.0_1', 'ts_Own.or.rent.accommodation.lived.in...Instance.0_4', 'ts_Hair.colour..natural..before.greying....Instance.0_3', 'ts_Age.at.last.live.birth...Instance.0_Lower.third', 'ts_Illnesses.of.adopted.mother...Instance.0_1', 'ts_Duration.of.heavy.DIY...Instance.0_3', 'ts_Age.at.first.live.birth...Instance.0_Lower.third', 'ts_Age.stopped.smoking...Instance.0_Lower.third', 'ts_Duration.of.heavy.DIY...Instance.0_6', 'ts_Illnesses.of.mother...Instance.0_11', 'ts_Duration.of.light.DIY...Instance.0_5', 'ts_Age.of.primiparous.women.at.birth.of.child...Instance.0_Lower.third', 'ts_Duration.of.other.exercises...Instance.0_6', 'ts_Oily.fish.intake...Instance.0_4', 'ts_Light.smokers..at.least.100.smokes.in.lifetime...Instance.0_1', 'ts_Non.butter.spread.type.details...Instance.0_7', 'ts_Illnesses.of.adopted.siblings...Instance.0_neg27', 'ts_Headaches.for.3..months...Instance.0_1', 'ts_Frequency.of.travelling.from.home.to.job.workplace...Instance.0_Upper.third', 'ts_Duration.of.light.DIY...Instance.0_1', 'ts_Illnesses.of.adopted.siblings...Instance.0_5', 'ts_Usual.side.of.head.for.mobile.phone.use...Instance.0_3', 'ts_Vitamin.and.mineral.supplements...Instance.0_5', 'ts_Guilty.feelings...Instance.0_1', 'ts_Illnesses.of.adopted.siblings...Instance.0_10', 'ts_Ever.had.same.sex.intercourse...Instance.0_1', 'ts_Neck.shoulder.pain.for.3..months...Instance.0_0', 'ts_Daytime.dozing...sleeping...Instance.0_3', 'ts_Leisure.social.activities...Instance.0_3', 'ts_Frequency.of.tenseness...restlessness.in.last.2.weeks...Instance.0_2', 'ts_Own.or.rent.accommodation.lived.in...Instance.0_6', 'ts_Lamb.mutton.intake...Instance.0_4', 'ts_Guilty.feelings...Instance.0_0', 'ts_Mineral.and.other.dietary.supplements...Instance.0_neg7', 'ts_Leisure.social.activities...Instance.0_5', 'ts_Average.monthly.spirits.intake...Instance.0_Upper.third', 'ts_Frequency.of.light.DIY.in.last.4.weeks...Instance.0_5', 'ts_Age.asthma.diagnosed...Instance.0_Middle.third', 'ts_Ethnic.background...Instance.0_1001', 'ts_Difference.in.mobile.phone.use.compared.to.two.years.previously...Instance.0_0', 'ts_Illnesses.of.adopted.siblings...Instance.0_12', 'ts_Lifetime.number.of.same.sex.sexual.partners...Instance.0_Upper.third', 'ts_Illnesses.of.adopted.mother...Instance.0_11', 'ts_Adopted.as.a.child...Instance.0_0', 'ts_Comparative.height.size.at.age.10...Instance.0_1', 'ts_Illnesses.of.father...Instance.0_4', 'ts_Oily.fish.intake...Instance.0_2', 'ts_Frequency.of.other.exercises.in.last.4.weeks...Instance.0_6', 'ts_Year.immigrated.to.UK..United.Kingdom....Instance.0_Middle.third', 'ts_Worrier...anxious.feelings...Instance.0_0', 'ts_Age.at.bilateral.oophorectomy..both.ovaries.removed....Instance.0_Upper.third', 'ts_Adopted.as.a.child...Instance.0_1', 'ts_Fracture.resulting.from.simple.fall...Instance.0_0', 'ts_Hot.drink.temperature...Instance.0_2', 'ts_How.are.people.in.household.related.to.participant...Instance.0_7', 'ts_Frequency.of.strenuous.sports.in.last.4.weeks...Instance.0_6', 'ts_Ever.had.same.sex.intercourse...Instance.0_0', 'ts_Bilateral.oophorectomy..both.ovaries.removed....Instance.0_1', 'ts_Worrier...anxious.feelings...Instance.0_1', 'ts_Age.at.menopause..last.menstrual.period....Instance.0_Lower.third', 'ts_Lifetime.number.of.same.sex.sexual.partners...Instance.0_Middle.third', 'ts_Miserableness...Instance.0_0', 'ts_Non.butter.spread.type.details...Instance.0_8', 'ts_Frequency.of.walking.for.pleasure.in.last.4.weeks...Instance.0_5', 'ts_Illnesses.of.adopted.siblings...Instance.0_neg17', 'ts_How.are.people.in.household.related.to.participant...Instance.0_3', 'ts_Miserableness...Instance.0_1', 'ts_Skin.colour...Instance.0_1', 'ts_Duration.walking.for.pleasure...Instance.0_2', 'ts_Blood.clot..DVT..bronchitis..emphysema..asthma..rhinitis..eczema..allergy.diagnosed.by.doctor...Instance.0_neg7', 'ts_Frequency.of.heavy.DIY.in.last.4.weeks...Instance.0_2', 'ts_Ever.used.hormone.replacement.therapy..HRT....Instance.0_1', 'ts_Hands.free.device.speakerphone.use.with.mobile.phone.in.last.3.month...Instance.0_4', 'ts_Current.employment.status...Instance.0_6', 'ts_Illnesses.of.father...Instance.0_12', 'ts_Hands.free.device.speakerphone.use.with.mobile.phone.in.last.3.month...Instance.0_3', 'ts_Plays.computer.games...Instance.0_1', 'ts_Difficulty.not.smoking.for.1.day...Instance.0_1', 'ts_Illnesses.of.adopted.father...Instance.0_13', 'ts_Illnesses.of.mother...Instance.0_12', 'ts_Age.at.bilateral.oophorectomy..both.ovaries.removed....Instance.0_Lower.third', 'ts_Pork.intake...Instance.0_4', 'ts_Illnesses.of.adopted.siblings...Instance.0_6', 'ts_Non.oily.fish.intake...Instance.0_5', 'ts_Drive.faster.than.motorway.speed.limit...Instance.0_2', 'ts_Sleep.duration...Instance.0', 'ts_Age.started.hormone.replacement.therapy..HRT....Instance.0_Upper.third', 'ts_Frequency.of.tiredness...lethargy.in.last.2.weeks...Instance.0_2', 'ts_Number.of.days.week.walked.10..minutes...Instance.0_Lower.third', 'ts_Years.since.last.cervical.smear.test...Instance.0_Upper.third', 'ts_Age.first.had.sexual.intercourse...Instance.0_neg2', 'ts_Gestational.diabetes.only...Instance.0_1', 'ts_Age.last.used.hormone.replacement.therapy..HRT....Instance.0_Lower.third', 'ts_Illnesses.of.siblings...Instance.0_12', 'ts_Time.from.waking.to.first.cigarette...Instance.0_5', 'ts_Duration.walking.for.pleasure...Instance.0_7', 'ts_Coffee.type...Instance.0_1', 'ts_Ethnic.background...Instance.0_3003', 'ts_Ever.taken.oral.contraceptive.pill...Instance.0_0', 'ts_Milk.type.used...Instance.0_5', 'ts_Illnesses.of.adopted.siblings...Instance.0_4', 'ts_Types.of.physical.activity.in.last.4.weeks...Instance.0_5', 'ts_Duration.of.walks...Instance.0_Lower.third', 'ts_Hot.drink.temperature...Instance.0_neg2', 'ts_Hearing.difficulty.problems...Instance.0_99', 'ts_Handedness..chirality.laterality....Instance.0_1', 'ts_Illnesses.of.adopted.siblings...Instance.0_3', 'ts_Facial.pains.for.3..months...Instance.0_0', 'ts_Milk.type.used...Instance.0_2', 'ts_Vitamin.and.mineral.supplements...Instance.0_2', 'ts_Back.pain.for.3..months...Instance.0_0', 'ts_Mouth.teeth.dental.problems...Instance.0_1', 'ts_Comparative.body.size.at.age.10...Instance.0_2', 'ts_Ease.of.skin.tanning...Instance.0_4', 'ts_Average.monthly.red.wine.intake...Instance.0_Upper.third', 'ts_Illnesses.of.adopted.mother...Instance.0_10', 'ts_Nervous.feelings...Instance.0_0', 'ts_Job.involves.heavy.manual.or.physical.work...Instance.0_4', 'ts_Ethnic.background...Instance.0_2', 'ts_Skin.colour...Instance.0_4', 'ts_Weight.change.compared.with.1.year.ago...Instance.0_2', 'ts_Illnesses.of.adopted.mother...Instance.0_6', 'ts_Knee.pain.for.3..months...Instance.0_0', 'ts_Poultry.intake...Instance.0_5', 'ts_Hair.colour..natural..before.greying....Instance.0_1', 'ts_Ethnic.background...Instance.0_2001', 'ts_Illnesses.of.adopted.father...Instance.0_1', 'ts_Frequency.of.light.DIY.in.last.4.weeks...Instance.0_4', 'ts_Ever.had.cervical.smear.test...Instance.0_0', 'ts_Time.spent.using.computer...Instance.0', 'ts_Lifetime.number.of.sexual.partners...Instance.0_Middle.third', 'ts_Duration.of.other.exercises...Instance.0_1', 'ts_Ethnic.background...Instance.0_2004', 'ts_Ethnic.background...Instance.0_3004', 'ts_Illnesses.of.adopted.mother...Instance.0_5', 'ts_Duration.of.heavy.DIY...Instance.0_1', 'ts_Mineral.and.other.dietary.supplements...Instance.0_2', 'ts_Frequency.of.depressed.mood.in.last.2.weeks...Instance.0_2', 'ts_Morning.evening.person..chronotype....Instance.0_3', 'ts_Duration.of.strenuous.sports...Instance.0_7', 'ts_Number.of.days.week.of.moderate.physical.activity.10..minutes...Instance.0', 'ts_Nervous.feelings...Instance.0_1', 'ts_Age.at.hysterectomy...Instance.0_Upper.third', 'ts_Qualifications...Instance.0_5', 'ts_Current.tobacco.smoking...Instance.0_2', 'ts_Illnesses.of.adopted.siblings...Instance.0_13', 'ts_Morning.evening.person..chronotype....Instance.0_4', 'ts_Average.monthly.intake.of.other.alcoholic.drinks...Instance.0_Upper.third', 'ts_Illness..injury..bereavement..stress.in.last.2.years...Instance.0_3', 'ts_Number.of.cigarettes.currently.smoked.daily..current.cigarette.smokers....Instance.0_Lower.third', 'ts_Pork.intake...Instance.0_5', 'ts_Had.menopause...Instance.0_2', 'ts_Country.of.birth..UK.elsewhere....Instance.0_2', 'ts_Illnesses.of.adopted.father...Instance.0_3', 'ts_Age.at.bilateral.oophorectomy..both.ovaries.removed....Instance.0_Middle.third', 'ts_Illnesses.of.adopted.siblings...Instance.0_1', 'ts_Duration.of.heavy.DIY...Instance.0_2', 'ts_Average.monthly.champagne.plus.white.wine.intake...Instance.0_Upper.third', 'ts_Frequency.of.friend.family.visits...Instance.0_5', 'ts_Illnesses.of.adopted.mother...Instance.0_3', 'ts_Vitamin.and.mineral.supplements...Instance.0_4']
mrmrVeto = mrmr[int(0.8 * len(mrmr)):]
print(len(dfTest))
dfTrain.drop(columns=[col for col in dfTrain.columns if col in mrmrVeto], inplace=True)
print(len(dfTest))


118376
118376


In [8]:
def to_binary(val):
    if val in [True, 'True', 'true', 1]:
        return 1
    elif val in [False, 'False', 'false', 0]:
        return 0
    return val  # Leave everything else unchanged
to_binary_vectorized = np.vectorize(to_binary)

In [9]:
def apply_binary_conversion(df):
    for col in df.columns:
        unique_vals = set(df[col].dropna().unique())
        if unique_vals.issubset({True, False, 1, 0, 'True', 'False', 'true', 'false'}):
            df[col] = to_binary_vectorized(df[col])
    return df


In [10]:
cleaned_model_to_csvs = {
    "Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history_Lifestyle and environment_oneDrop_qrisk": [
        "Sociodemographics",
        "Health and medical history",
        "Sex-specific factors",
        "Early life factors",
        "Family history",
        "Lifestyle and environment",
        "oneDrop",
        "qrisk"
    ],
    "Sociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_Lifestyle_and_environment_Psychosocial_factors_oneDrop_qrisk": [
        "Sociodemographics",
        "Health and medical history",
        "Sex-specific factors",
        "Early life factors",
        "Family history",
        "Lifestyle and environment",
        "Psychosocial factors",
        "oneDrop",
        "qrisk"
    ],
}

In [11]:
from collections import defaultdict
import re

def get_columns_by_model_category(model_to_categories, df):
    """
    Returns a dict:  model_key  ->  list of feature columns.
    A feature column is selected if it contains ANY of that model’s
    high-level category strings (robust to stray spaces / case).
    """
    matched = defaultdict(list)

    cols = [c.strip() for c in df.columns]

    for model_key, categories in model_to_categories.items():
        pat = re.compile(rf"({'|'.join(map(re.escape, categories))})", flags=re.I)

        for col in cols:
            'get rid of two sex variables and one age'
            if col.startswith(("category_", "oneDrop_","qrisk_")) and pat.search(col):
                matched[model_key].append(col)

    return dict(matched)


In [12]:
def modelTrainer(cleaned_model_to_csvs, dfTrain, dfTest):
    import time
    dfTrain = dfTrain[dfTrain["CVD_followup"]>0]
    dfTest = dfTest[dfTest["CVD_followup"]>0]
    print(">>> Starting model training process...")
    start_time = time.time()

    # 0) strip stray spaces
    print(">>> Stripping stray column name spaces...")
    dfTrain.columns = dfTrain.columns.str.strip()
    dfTest.columns = dfTest.columns.str.strip()

    print(">>> Applying binary conversion where applicable...")
    dfTrain = apply_binary_conversion(dfTrain)
    dfTest = apply_binary_conversion(dfTest)

    # 1) build mapping model → cols
    print(">>> Building feature mappings from category metadata...")
    model_columns = get_columns_by_model_category(cleaned_model_to_csvs, dfTrain)
    print(">>> Models found:", list(model_columns.keys()))

    all_data = {}  # to hold train/test splits

    for model_index, (model_name, feature_cols) in enumerate(model_columns.items(), 1):
        print(f"\n>> [{model_index}/{len(model_columns)}] Training model: {model_name}")
        print(f"   >> Total features: {len(feature_cols)}")

        outcomes = ['CVD_status', 'CVD_followup', 'clinicalrisk_Age.at.recruitment']

        try:
            trainFrame = dfTrain.loc[:, feature_cols + outcomes].copy()
            testFrame = dfTest.loc[:, feature_cols + outcomes].copy()
        except KeyError as e:
            print(f"   !! Missing column(s): {e}. Skipping model.")
            continue

        # Check columns before proceeding
        print("   >> Columns before cleaning:", len(trainFrame.columns))

        # drop low-variance & non-numeric columns
        var = trainFrame.var()
        low_var = [c for c, v in var.items() if v < 1e-4 and c not in ('CVD_status', 'CVD_followup')]
        print(f"   >> Dropping {len(low_var)} low-variance columns...")
        trainFrame.drop(columns=low_var, inplace=True)
        testFrame.drop(columns=low_var, inplace=True, errors='ignore')

        non_num = trainFrame.select_dtypes(exclude='number').columns
        if len(non_num):
            print(f"   >> Dropping {len(non_num)} non-numeric columns...")
        trainFrame.drop(columns=non_num, inplace=True)
        testFrame.drop(columns=non_num, inplace=True, errors='ignore')
        
        print(f"Number of features ",{len(trainFrame.columns)})

        print("   >> Final columns for training:", len(trainFrame.columns))

        # split features and target
        X_train = trainFrame.drop(columns=['CVD_status', 'CVD_followup'])
        X_test = testFrame.drop(columns=['CVD_status', 'CVD_followup'])
        
        print(X_test['clinicalrisk_Age.at.recruitment'])

        y_train = np.array([(bool(e), t) for e, t in zip(trainFrame['CVD_status'], trainFrame['CVD_followup'])],
                           dtype=[('event', '?'), ('time', '<f8')])
        y_test = np.array([(bool(e), t) for e, t in zip(testFrame['CVD_status'], testFrame['CVD_followup'])],
                          dtype=[('event', '?'), ('time', '<f8')])
                # run grid search
        print("   >> Starting grid search for optimal alpha...")
        def c_index_scorer(est, X, y):
            return concordance_index_censored(y['event'], y['time'], est.predict(X))[0]

        gs = GridSearchCV(CoxPHSurvivalAnalysis(), {'alpha': [1e-4, 1e-3, 1e-2, 1e-1, 1.0]},
                          scoring=c_index_scorer, cv=5, verbose=2)
        gs.fit(X_train, y_train)
        best = gs.best_estimator_
        print("   >> Best alpha found:", gs.best_params_['alpha'])

        # evaluate
        print("   >> Evaluating on test set...")
        preds = best.predict(X_test)
        cidx = concordance_index_censored(y_test['event'], y_test['time'], preds)[0]
        print(f"   >> Test C-index: {cidx:.4f}")

        # save model
        safe = model_name.replace(' ', '_').replace('/', '_')
        fname = f"MRMR_COX_FULL_MODEL_+QRISK{safe}.pkl"
        with open(fname, 'wb') as f:
            pickle.dump(best, f)

        print(f"   >> Uploading model {fname} to DNAnexus...")
        res = subprocess.run(['dx', 'upload', fname, '--path', f"SADIQ/21 models/{fname}"],
                             capture_output=True, text=True)
        print("   >> Upload stdout:", res.stdout.strip())

        # save train/test splits
        all_data[model_name] = (trainFrame, testFrame)
        print(f"   >> Model '{model_name}' training complete.")

    print(f"\n>>> All models processed in {time.time() - start_time:.2f} seconds.")
    return all_data


In [ ]:
all_model_data = modelTrainer(cleaned_model_to_csvs, dfTrain, dfTest)

>>> Starting model training process...
>>> Stripping stray column name spaces...
>>> Applying binary conversion where applicable...
>>> Building feature mappings from category metadata...
>>> Models found: ['Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history_Lifestyle and environment_oneDrop_qrisk', 'Sociodemographics_Health_and_medical_history_Sex-specific_factors_Early_life_factors_Family_history_Lifestyle_and_environment_Psychosocial_factors_oneDrop_qrisk']

>> [1/2] Training model: Sociodemographics_Health and medical history_Sex-specific factors_Early life factors_Family history_Lifestyle and environment_oneDrop_qrisk
   >> Total features: 929
   >> Columns before cleaning: 932
   >> Dropping 6 low-variance columns...
Number of features  {926}
   >> Final columns for training: 926
0         48
1         65
2         50
3         63
4         45
          ..
120490    67
120491    52
120492    60
120493    57
120494    63
Name: clini

/opt/conda/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:173: RuntimeWarning: overflow encountered in exp
  risk_set += np.exp(xw[k])
/opt/conda/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:170: RuntimeWarning: overflow encountered in exp
  risk_set2 += np.exp(xw[k])


[CV] END .......................................alpha=0.0001; total time=39.5min


/opt/conda/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:173: RuntimeWarning: overflow encountered in exp
  risk_set += np.exp(xw[k])
/opt/conda/lib/python3.11/site-packages/sksurv/linear_model/coxph.py:170: RuntimeWarning: overflow encountered in exp
  risk_set2 += np.exp(xw[k])


In [ ]:
print("done")